# Capella API: Task and Status Check Example

In [ ]:
# Required libraries:
# requests
# json

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [74]:
import requests
import json
from datetime import datetime, timedelta

# Capella API endpoints
URL = 'https://api.capellaspace.com'
token = '/token'
task = '/task'
collects_list = '/collects/list/'

#Load username and password
with open('../credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
username

'david.hemphill@capellaspace.com'

### Get and Print Access Token

In [75]:
#Get the token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# Print the token
print("Access Token: " + accesstoken)

headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}

Access Token: eyJraWQiOiJCeFdcL0tmZ0QzK3pwVlRBSFZ2NlBnNXRtWjVpOXRYTm8zaDkycFwvT0VONXc9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJhMTkzMWQ5ZS0xNTY0LTQ2ZDUtYjEwZi1hZGE0NDQyYWNlZDQiLCJldmVudF9pZCI6IjM0OTQzNmQxLWMzNDctNGMzZS04NjIzLTBjZTRkYmU5ZDQwZiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE2NTE2MjgwNjMsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy13ZXN0LTIuYW1hem9uYXdzLmNvbVwvdXMtd2VzdC0yXzJmNFBGWThoOCIsImV4cCI6MTY1MTYzMTY2MywiaWF0IjoxNjUxNjI4MDYzLCJqdGkiOiI1NmYzMzk0Ny00NzY1LTQ2Y2ItYTI3NS1jZGJmOGY2MmM2ZWIiLCJjbGllbnRfaWQiOiIxMGlxZTdiZmp2a2lhMnBzdG9vNzhqaWo5MSIsInVzZXJuYW1lIjoiYzJlYTg4MzEtNmFjYy00ZWM0LWFmZGQtNWUxYjU0N2U5ZWYxIn0.GMutttfNF6w4bimQQhpeQScFsVAzsYmD3y99S9QVK8ZwveaJ-efeV4M-O-cmb_n54M6K3gdbXR42eGzVagT43eC4Qzkq9OzRlEneDPw-8J0SXGsV2FX5Zf8aTnn9AoaP8TCaJEjx1u04uxW41asFniYPg6Efk0QguuU1r2zEiYbMV-LIX2fRUPl_dtuiuPKiOhByMvoakKuHRc46VTsgJR5gVSYsZLoKqA-eAy3iDDhFqJstyFRO_2wVgfUFiDwqXlggZ_CdPlN01jNpykzUHt9V4LpFGFExgAheu7SBFNnHu1OGftsQ1TnzqIE2TvkZ_Bt

### Compute open and close dates

In [89]:
# helper function to calcaulate and format window open/close
# just run this and move on as it's called later on

def compute_window_open(window_open_date, window_close_days):
    window_close_date = window_open_date + timedelta(days=window_close_days)
    wo_str = datetime.strftime(window_open_date, "%Y-%m-%dT%H:%M:%S.%fZ")
    wc_str = datetime.strftime(window_close_date, "%Y-%m-%dT%H:%M:%S.%fZ")
    return wo_str, wc_str

### Set key variables for tasking

In [91]:
# lon_lat = [4.449528, 51.894330]   #Required
lon_lat = [-46.290314,-23.969401]   #Required
task_request_name = "Rotterdam"   #Required
task_request_description = "For tasking example" #Optional - displays in task status page on console
collectMode = "spotlight" #Required: spotlight, sliding_spotlight, stripmap
collectionTier = "1_day" #Required: 1_day, 3_day, or 7_day
# IMPORTANT: The window open/close must match or be larger than the collectionTier days
# e.g., 1_day > 24 hours. 3_day > 3 days, etc.

# date object for the window_open. Could be now or a future date. days=0 is now    
window_open_date = datetime.now() + timedelta(days=0)

# number of days past the open date to specifiy for window_close
window_duration = 1    

# Test and show output
wo_str, wc_str = compute_window_open(window_open_date, window_duration)
print("Window Open : %s" % wo_str )
print("Window Close: %s" % wc_str )
window_open = wo_str
window_close = wc_str

# Add analytics by including a processingConfig property. The following adds Vessel Detection
#   "processingConfig": {"productTypes": ["VS"]},

Window Open : 2022-05-03T20:41:05.935308Z
Window Close: 2022-05-04T20:41:05.935308Z


### Create the tasking payload

In [92]:
payload = {
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": lon_lat
      },
      "properties": {
        "taskingrequestName": "%s" % task_request_name,
        "taskingrequestDescription": "%s" % task_request_description,
        "windowOpen": "%s" % window_open,
        "windowClose": "%s" % window_close,
        "collectionTier": "%s" % collectionTier,
        "collectConstraints": {
          "collectMode": "%s" % collectMode
        },
        "preApproval": True
      }
    }
payload

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-46.290314, -23.969401]},
 'properties': {'taskingrequestName': 'Rotterdam',
  'taskingrequestDescription': 'For tasking example',
  'windowOpen': '2022-05-03T20:41:05.935308Z',
  'windowClose': '2022-05-04T20:41:05.935308Z',
  'collectionTier': '1_day',
  'collectConstraints': {'collectMode': 'spotlight'},
  'preApproval': True}}

### Post the task

In [93]:
call_url = URL + task
print("POST to %s" % call_url)
r = requests.post(call_url, json=payload, headers=headers)
response = r.json()
p(response)

POST to https://api.capellaspace.com/task
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -46.290314,
      -23.969401
    ]
  },
  "properties": {
    "submissionTime": "2022-05-04T01:41:31.972Z",
    "taskingrequestId": "abbddade-888b-4f03-8d55-b08365cac4dd",
    "taskingrequestName": "Rotterdam",
    "taskingrequestDescription": "For tasking example",
    "orgId": "64405d54-9fa6-474e-85fb-7862a5cc3891",
    "userId": "c2ea8831-6acc-4ec4-afdd-5e1b547e9ef1",
    "repeatrequestId": null,
    "windowOpen": "2022-05-03T20:41:05.935Z",
    "windowDuration": 86400,
    "windowClose": "2022-05-04T20:41:05.935Z",
    "collectionTier": "1_day",
    "archiveHoldback": "none",
    "statusHistory": [
      {
        "time": "2022-05-04T01:41:31.972Z",
        "code": "received",
        "message": "Request created"
      }
    ],
    "collectConstraints": {
      "lookDirection": "either",
      "ascDsc": "either",
      "orbitalPlanes": [],
      "localTim

### Check tasking status

In [94]:
# Get the taskingrequestId
taskingrequestId = response["properties"]["taskingrequestId"]
taskingrequestId

'abbddade-888b-4f03-8d55-b08365cac4dd'

In [107]:
# Check the status - run this as needed to watch progression of status
call_url = URL + task + "/" + taskingrequestId
print("Calling %s" % call_url)
r = requests.get(call_url, headers=headers)
response = r.json()

#Current status is at the front of the array
statusHistory = response["properties"]["statusHistory"]
current_status = statusHistory[0]
current_status

Calling https://api.capellaspace.com/task/abbddade-888b-4f03-8d55-b08365cac4dd


{'time': '2022-05-04T01:46:07.215Z',
 'code': 'accepted',
 'message': 'Request can be completely satisfied during validity window.'}

### Get the collect id

In [106]:
# After a task is accepted, it is assigned a collect id
# The collect id becomes the primary reference id to find 
# it in catalog after it is collected - and to place an order for download

call_url = URL + collects_list + taskingrequestId
print("Calling %s" % call_url)
r = requests.get(call_url, headers=headers)
response = r.json()
collectId = "No collect id available until task is accepted."
if len(response) > 0: 
    collectId = response[0]['collectId'] # only 1 collect expected
collectId

Calling https://api.capellaspace.com/collects/list/abbddade-888b-4f03-8d55-b08365cac4dd


'f67a23a2-fb60-4794-b887-c1139e2b9d34'